In [1]:
!pip install arxiv

  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
   ---------------------------------------- 0.0/81.1 kB ? eta -:--:--
   ---------------------------------------- 0.0/81.1 kB ? eta -:--:--
   ---------------------------------------- 0.0/81.1 kB ? eta -:--:--
   ---------------------------------------- 81.1/81.1 kB 2.3 MB/s eta 0:00:00
  Created wheel for sgmllib3k: filename=sgmllib3k-1.0.0-py3-none-any.whl size=6061 sha256=c75a298e8bad891821dd3de877e08ff8b134f9e52565ab26e15cdf715ebe6685
  Stored in directory: c:\users\arung\appdata\local\pip\cache\wheels\03\f5\1a\23761066dac1d0e8e683e5fdb27e12de53209d05a4a37e6246
Successfully built sgmllib3k


In [2]:
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

In [12]:
# WikipediaAPIWrapper this will intract with wikipedia and we can set how results we want

api_wrapper=WikipediaAPIWrapper(top_k_results=1,doc_content_chars_max=200)
wiki=WikipediaQueryRun(api_wrapper=api_wrapper)

In [13]:
wiki.name

'wikipedia'

In [8]:
from langchain_community.document_loaders import WebBaseLoader
from langchain.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

loader=WebBaseLoader("https://docs.smith.langchain.com/")
docs=loader.load()
documents=RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200).split_documents(docs)
embeddings=OpenAIEmbeddings()
vector_db=FAISS.from_documents(documents,embeddings)
retriever=vector_db.as_retriever()
retriever



VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x000001B7822080B0>)

In [9]:
# We already have 2 tool wikiedia and langsmith
from langchain.tools.retriever import create_retriever_tool # This will help to create QA within the langsmith document

retriever_tool=create_retriever_tool(retriever,"langsmith_search",
                      "search information about langsmith.for any question about lang")
retriever_tool.name

'langsmith_search'

In [10]:
# Arxiv Tool
# This will provoide all the research paper details we can make QA about research paper
from langchain_community.tools import ArxivQueryRun
from langchain_community.utilities import ArxivAPIWrapper

arxiv_wrapper=ArxivAPIWrapper(top_k_results=1,doc_content_chars_max=200)
arxiv=ArxivQueryRun(api_wrapper=arxiv_wrapper)
arxiv.name

'arxiv'

In [11]:
# we totally have 3 tools

In [14]:
tools=[wiki,retriever_tool,arxiv]
tools

[WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper(wiki_client=<module 'wikipedia' from 'c:\\Users\\arung\\anaconda3\\envs\\aiprojects\\Lib\\site-packages\\wikipedia\\__init__.py'>, top_k_results=1, lang='en', load_all_available_meta=False, doc_content_chars_max=200)),
 Tool(name='langsmith_search', description='search information about langsmith.for any question about lang', args_schema=<class 'langchain.tools.retriever.RetrieverInput'>, func=functools.partial(<function _get_relevant_documents at 0x000001B744E9CAE0>, retriever=VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x000001B7822080B0>), document_prompt=PromptTemplate(input_variables=['page_content'], template='{page_content}'), document_separator='\n\n'), coroutine=functools.partial(<function _aget_relevant_documents at 0x000001B744E9CB80>, retriever=VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstore

In [16]:
import os
from dotenv import load_dotenv
load_dotenv()

os.environ['OPENAI_API_KEY']=os.getenv("OPENAI_API_KEY")

from langchain_openai import ChatOpenAI 

llm = ChatOpenAI(temperature=0.5, model_name="gpt-3.5-turbo")

In [19]:
# We can create prompt in mulpe way one of the way is langchainhub which was provoided by langchain
from langchain import hub
prompt=hub.pull("hwchase17/openai-functions-agent")
prompt.messages

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='You are a helpful assistant')),
 MessagesPlaceholder(variable_name='chat_history', optional=True),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], template='{input}')),
 MessagesPlaceholder(variable_name='agent_scratchpad')]

In [21]:
#Agents will responsible for accessing the realtime data 
# Ex above we have 3 tool sequentialy whenever i ask any qs it will search one by one if it there then return ans
from langchain.agents import create_openai_tools_agent
agent=create_openai_tools_agent(llm=llm, tools=tools,prompt=prompt)

In [22]:
# When we want to excute agent we should use below
from langchain.agents import  AgentExecutor
agent_executor=AgentExecutor(agent=agent,tools=tools,verbose=True)
agent_executor

AgentExecutor(verbose=True, agent=RunnableMultiActionAgent(runnable=RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_to_openai_tool_messages(x['intermediate_steps']))
})
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], input_types={'chat_history': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]], 'agent_scratchpad': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]}, metadata={'lc_hub_owner': 'hwchase17', 'lc_hub_repo': 'openai-functions-agent', 'lc_hub_commit_hash': 'a165

In [23]:
agent_executor.invoke({'input':'tell me about langsmith'})



> Entering new AgentExecutor chain...

Invoking: `wikipedia` with `Langsmith`


No good Wikipedia Search Result was found
Invoking: `langsmith_search` with `{'query': 'Langsmith'}`


Getting started with LangSmith | 🦜️🛠️ LangSmith

Skip to main contentLangSmith API DocsSearchGo to AppQuick StartUser GuideTracingEvaluationProduction Monitoring & AutomationsPrompt HubProxyPricingSelf-HostingCookbookQuick StartOn this pageGetting started with LangSmithIntroduction​LangSmith is a platform for building production-grade LLM applications. It allows you to closely monitor and evaluate your application, so you can ship quickly and with confidence. Use of LangChain is not necessary - LangSmith works on its own!Install LangSmith​We offer Python and Typescript SDKs for all your LangSmith needs.PythonTypeScriptpip install -U langsmithyarn add langchain langsmithCreate an API key​To create an API key head to the setting pages. Then click Create API Key.Setup your environment​Shellexport LANGCHAIN_

{'input': 'tell me about langsmith',
 'output': 'LangSmith is a platform for building production-grade LLM (Large Language Model) applications. It allows you to closely monitor and evaluate your application, ensuring you can ship quickly and with confidence. LangSmith offers Python and Typescript SDKs for all your needs. You can create an API key, set up your environment, and log your first trace using LangSmith. \n\nLangSmith supports workflows at each stage of the LLM application lifecycle and provides features like tracing, evaluation, pricing models, self-hosting options, proxy capabilities, and a Prompt Hub for prompt management. \n\nFor more information, you can visit the LangSmith Cookbook, LangChain Python and TypeScript documentation, and join their Discord community for discussions. If you have specific questions or need assistance with private deployment or self-hosting, you can reach out to LangSmith at sales@langchain.dev.'}

In [24]:
# We got whole data 

agent_executor.invoke({'input':'What are skills necessory for data scitients?'})



> Entering new AgentExecutor chain...

Invoking: `wikipedia` with `Skills required for data scientists`


Page: Data science
Summary: Data science is an interdisciplinary academic field that uses statistics, scientific computing, scientific methods, processes, algorithms and systems to extract or extrapolData scientists require a diverse set of skills including statistics, scientific computing, scientific methods, processes, algorithms, and systems to extract and analyze data. These skills are essential for data scientists to perform their roles effectively.

> Finished chain.


{'input': 'What are skills necessory for data scitients?',
 'output': 'Data scientists require a diverse set of skills including statistics, scientific computing, scientific methods, processes, algorithms, and systems to extract and analyze data. These skills are essential for data scientists to perform their roles effectively.'}

In [25]:
agent_executor.invoke({'input':'What is the paper 1605.08386?'})



> Entering new AgentExecutor chain...

Invoking: `arxiv` with `{'query': '1605.08386'}`


Published: 2016-05-26
Title: Heat-bath random walks with Markov bases
Authors: Caprice Stanley, Tobias Windisch
Summary: Graphs on lattice points are studied whose edges come from a finite set of
alloThe paper with the identifier 1605.08386 is titled "Heat-bath random walks with Markov bases." It was published on May 26, 2016, and the authors are Caprice Stanley and Tobias Windisch. The paper discusses the study of graphs on lattice points where the edges are derived from a finite set of Markov bases.

> Finished chain.


{'input': 'What is the paper 1605.08386?',
 'output': 'The paper with the identifier 1605.08386 is titled "Heat-bath random walks with Markov bases." It was published on May 26, 2016, and the authors are Caprice Stanley and Tobias Windisch. The paper discusses the study of graphs on lattice points where the edges are derived from a finite set of Markov bases.'}